In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import os
import shutil
import pydicom
import itk
from pathlib import Path

In [2]:
df = pd.read_excel('/workspace/src/DLT/analysis/final_230404.xlsx')
df

,HID,OPDATE,DESCRIPTION,ACQUISITION_DATE_TIME,AIRWAY_DOUBLE_LUMEN,PATH,JUPYTER_PATH,FILE_CNT
0,7745599,2016-06-07 00:00:00,Chest PA,2016-06-04 15:45:10,DLT35.,D:\20230307\07745599\20160604,/workspace/src/DLT/20230307/07745599/20160604,4
1,7779206,2008-05-07 00:00:00,Chest PA,2008-05-06 16:50:34,DLT37.,D:\20230307\07779206\20080506,/workspace/src/DLT/20230307/07779206/20080506,4
2,8212838,2011-06-02 00:00:00,Chest PA,2011-06-01 13:06:42,DLT32.,D:\20230307\08212838\20110601,/workspace/src/DLT/20230307/08212838/20110601,1
3,8652241,2014-08-25 00:00:00,Chest PA,2014-08-24 19:17:09,DLT35.,D:\20230307\08652241\20140824,/workspace/src/DLT/20230307/08652241/20140824,4
4,8684332,2016-11-07 00:00:00,Chest PA,2016-11-03 16:26:57,DLT32.,D:\20230307\08684332\20161103,/workspace/src/DLT/20230307/08684332/20161103,1
...,...,...,...,...,...,...,...,...
15105,77759559,2016-12-20 00:00:00,Chest PA,2016-12-19 13:29:12,DLT37.,D:\20230307\77759559\20161219,/workspace/src/DLT/20230307/77759559/20161219,4
15106,78935440,2021-11-01 00:00:00,Chest PA,2021-10-31 12:23:34,DLT37.,D:\20230307\78935440\20211031,/workspace/src/DLT/20230307/78935440/20211031,2
15107,78984581,2021-03-19 00:00:00,Chest PA,2021-03-17 17:49:29,DLT32.,D:\20230307\78984581\20210317,/workspace/src/DLT/20230307/78984581/20210317,4
15108,79976778,2020-09-07 00:00:00,Chest PA - AI CAD,2020-09-05 05:54:55,DLT32.,D:\20230307\79976778\20200905,/workspace/src/DLT/20230307/79976778/20200905,2


In [17]:
jupyter_path_list = df['JUPYTER_PATH'].tolist()
file_cnt_list = df['FILE_CNT'].tolist()
description_list = df['DESCRIPTION'].tolist()

In [12]:
file_list = os.listdir('/workspace/src/DLT/20230307/08212838/20110601')
file_list[0]

'1.2.392.200036.9125.9.0.252533546.3036676288.756119615.dcm'

In [18]:
dcm = []
for i in range(len(file_cnt_list)):
    folder_path = jupyter_path_list[i]
    file_list = os.listdir(folder_path)
    if file_cnt_list[i] == 1:
        dcm.append(file_list[0])
    else:
        cnt = 0
        dcm_o = []
        for j in range(len(file_list)):
            read_name = folder_path+'/'+file_list[j]
            ds = pydicom.read_file(read_name)
            if 'StudyDescription' in dir(ds):
                if description_list[i] == ds.StudyDescription:
                    dcm_o.append(file_list[j])
                    cnt += 1
        if cnt == 1:
            dcm.append(dcm_o[0])
        else:
            dcm.append("find")

In [19]:
df['DCM'] = dcm
df

,HID,OPDATE,DESCRIPTION,ACQUISITION_DATE_TIME,AIRWAY_DOUBLE_LUMEN,PATH,JUPYTER_PATH,FILE_CNT,DCM
0,7745599,2016-06-07 00:00:00,Chest PA,2016-06-04 15:45:10,DLT35.,D:\20230307\07745599\20160604,/workspace/src/DLT/20230307/07745599/20160604,4,1.3.46.670589.26.802065.4.20160604.154535.7684...
1,7779206,2008-05-07 00:00:00,Chest PA,2008-05-06 16:50:34,DLT37.,D:\20230307\07779206\20080506,/workspace/src/DLT/20230307/07779206/20080506,4,1.3.46.670589.26.802065.4.20080506.165056.9467...
2,8212838,2011-06-02 00:00:00,Chest PA,2011-06-01 13:06:42,DLT32.,D:\20230307\08212838\20110601,/workspace/src/DLT/20230307/08212838/20110601,1,1.2.392.200036.9125.9.0.252533546.3036676288.7...
3,8652241,2014-08-25 00:00:00,Chest PA,2014-08-24 19:17:09,DLT35.,D:\20230307\08652241\20140824,/workspace/src/DLT/20230307/08652241/20140824,4,1.3.46.670589.26.802065.4.20140824.191757.8455...
4,8684332,2016-11-07 00:00:00,Chest PA,2016-11-03 16:26:57,DLT32.,D:\20230307\08684332\20161103,/workspace/src/DLT/20230307/08684332/20161103,1,1.3.46.670589.26.802065.4.20161103.162728.1354...
...,...,...,...,...,...,...,...,...,...
15105,77759559,2016-12-20 00:00:00,Chest PA,2016-12-19 13:29:12,DLT37.,D:\20230307\77759559\20161219,/workspace/src/DLT/20230307/77759559/20161219,4,1.3.46.670589.26.802065.4.20161219.132938.1514...
15106,78935440,2021-11-01 00:00:00,Chest PA,2021-10-31 12:23:34,DLT37.,D:\20230307\78935440\20211031,/workspace/src/DLT/20230307/78935440/20211031,2,find
15107,78984581,2021-03-19 00:00:00,Chest PA,2021-03-17 17:49:29,DLT32.,D:\20230307\78984581\20210317,/workspace/src/DLT/20230307/78984581/20210317,4,1.3.46.670589.26.802065.4.20210317.175001.7284...
15108,79976778,2020-09-07 00:00:00,Chest PA - AI CAD,2020-09-05 05:54:55,DLT32.,D:\20230307\79976778\20200905,/workspace/src/DLT/20230307/79976778/20200905,2,find


In [21]:
df.to_excel('/workspace/src/DLT/analysis/final_dcm.xlsx')

In [20]:
df[df['DCM'] == 'find']

,HID,OPDATE,DESCRIPTION,ACQUISITION_DATE_TIME,AIRWAY_DOUBLE_LUMEN,PATH,JUPYTER_PATH,FILE_CNT,DCM
10,8994116,2010-03-22 00:00:00,Chest PA,2010-03-18 12:38:33,DLT37.,D:\20230307\08994116\20100318,/workspace/src/DLT/20230307/08994116/20100318,2,find
17,9856231,2011-04-19 00:00:00,Chest PA,2011-04-18 17:12:57,DLT35.,D:\20230307\09856231\20110418,/workspace/src/DLT/20230307/09856231/20110418,4,find
22,10051098,2017-08-08 00:00:00,Chest PA,2017-08-06 15:18:04,DLT37.,D:\20230307\10051098\20170806,/workspace/src/DLT/20230307/10051098/20170806,4,find
28,10601967,2008-04-14 00:00:00,Chest PA,2008-04-07 10:25:57,DLT37.,D:\20230307\10601967\20080407,/workspace/src/DLT/20230307/10601967/20080407,2,find
36,10917794,2019-09-03 00:00:00,Chest PA,2019-09-02 14:46:47,DLT37.,D:\20230307\10917794\20190902,/workspace/src/DLT/20230307/10917794/20190902,4,find
...,...,...,...,...,...,...,...,...,...
15081,75420848,2017-04-21 00:00:00,Chest PA,2017-04-20 14:26:41,DLT37.,D:\20230307\75420848\20170420,/workspace/src/DLT/20230307/75420848/20170420,4,find
15086,75597731,2022-01-18 00:00:00,Chest PA - AI CAD,2022-01-17 16:41:23,DLT35.,D:\20230307\75597731\20220117,/workspace/src/DLT/20230307/75597731/20220117,5,find
15088,75736303,2010-03-11 00:00:00,Chest PA,2010-03-10 17:02:35,DLT37.,D:\20230307\75736303\20100310,/workspace/src/DLT/20230307/75736303/20100310,3,find
15106,78935440,2021-11-01 00:00:00,Chest PA,2021-10-31 12:23:34,DLT37.,D:\20230307\78935440\20211031,/workspace/src/DLT/20230307/78935440/20211031,2,find
